# Step Testing Lab Assignment
## Task 1: Plot Experimental Data and Estimate Parameters from 1st Order Model

In [ ]:
import pandas as pd

# parameters
P1 = 200
U1 = 50

#file location
file_path = "data.csv" # my data file from the testing experiment is in the same folder as this notebook

# get data
data = pd.read_csv(file_path, index_col = "Time")

#display step test data
display(data)

#plot data
data.plot(y=["T1", "T2"], 
          title=f"{P1=}, {U1=}",
          xlabel="seconds", 
          ylabel="deg C", 
          grid=True, 
          figsize=(8, 2.5)
         )

data.plot(y=["Q1", "Q2"],
          title=f"{P1=}, {U1=}",
          xlabel="seconds", 
          ylabel="% of full range", 
          grid=True, 
          figsize=(8, 1.5),
          ylim=(0, 100)
         )

### Parameter Estimation
Gain - if the system is at steady state then: $$0 = a \bar{x} + b \bar{u}$$
so $$\bar{x}=-\frac{b}{a}\bar{u}$$
where the gain K is $$K=-\frac{b}{a}$$

in this case, the steady state temperature is ~55.74&deg;C and the ambient is 23&deg;C so $\bar{x} = 32.74C$

and $\bar{u}=50$ % corresponding to the heater output

thus $K=.6548$ &deg;C/%



Time constant - time it takes to get to 63.2% of the way to the final steady state value: $$.623*(55.74-23)+23=43.98C$$

In [ ]:
around44 = data[data['T1'].between(43.8,44.2)]
display(around44)

This temp. is between 176s and 177s so:
$\tau$ = 176.5 s

Ambient temperature - The steady state temperature: 23&deg;C

## Task 2: Plot Analytical Solution and Adjust Parameters for a Good Fit
### Initial Plot

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# parameter values
T_amb = 23 # °C
x0 = 0 # °C
K = 0.6548 # °C/% U1
tau = 176.5 # sec

# known input
U1 = 50 # %

# compute analytical solution
t = data.index
x = x0*np.exp(-t/tau) + (1 - np.exp(-t/tau))*K*U1

# plotting solution for T1
T1 = x + T_amb
plt.plot(t, T1, t, data["T1"])
plt.legend(["T1 predicted", "T1 measured"])

# dress up the plot
plt.title(f"{x0=}, {K=}, {tau=}")
plt.xlabel("seconds")
plt.ylabel("deg C")
plt.grid(True)

### Measuring Fit

In [ ]:
# Measuring the fit of the model

# re-defining things
t = data.index
T1 = data['T1'].values

# redefining parameters
T_amb = 23 # deg C
x0 = 0 # °C

# the model 'p'
K = 0.6548 # °C/% U1
tau = 176.5 # sec

def compare(p, plot=False): # a fuction that takes values for K and tau and then compares the model to the data.
    # if plot is true it will also do some graphs to show
    
    K, tau = p

    # model
    T1_dev_initial = x0 # intial deviation temperature - 0 if at ambient
    T1_dev_ss = K*U1 # steady state deviation value at U1 output (50% in this case)
    T1_dev = T1_dev_initial*np.exp(-t/tau) + (1 - np.exp(-t/tau))*T1_dev_ss # model temp. deviation
    T1_model = T1_dev + T_amb # model actual temp.

    # model mismatch calculated
    sse = sum((T1_model - T1)**2)
    sae = sum(np.abs(T1_model - T1))

    # visualization
    if plot:
        fig, ax = plt.subplots(4, 1, figsize=(10,10))
        ax[0].plot(t, T1, 'r', lw=2)
        ax[0].plot(t, T1_model, 'r--', lw=2)
        ax[0].axhline(T_amb)
        ax[0].axvline(0)
        ax[0].set_xlabel("time / seconds")
        ax[0].set_ylabel("temperture / °C")
        ax[0].set_title(f"Step Response (P1 = {P1}, U1 = {U1})")
        ax[0].grid(True)

        ax[1].plot(t, T1_model - T1)
        ax[1].axhline(0)
        ax[1].axvline(0)
        ax[1].fill_between(t, T1_model - T1, alpha=0.2)
        ax[1].set_title(f'Model Error')
        ax[1].set_xlabel("time / seconds")
        ax[1].set_ylabel("temperture / °C")
        ax[1].grid(True)

        ax[2].plot(t, (T1_model - T1)**2)
        ax[2].axhline(0)
        ax[2].axvline(0)
        ax[2].fill_between(t, (T1_model - T1)**2, alpha=0.2)
        ax[2].set_title(f'Sum of Squares = {sse:0.1f}')
        ax[2].set_xlabel("time / seconds")
        ax[2].set_ylabel("(temperture / °C)^2")
        ax[2].grid(True)

        ax[3].plot(t, np.abs(T1_model - T1))
        ax[3].axhline(0)
        ax[3].axvline(0)
        ax[3].fill_between(t, np.abs(T1_model - T1), alpha=0.2)
        ax[3].set_title(f'Sum of Absolute Values = {sae:0.1f}')
        ax[3].set_xlabel("time / seconds")
        ax[3].set_ylabel("temperture / °C")
        ax[3].grid(True)

        plt.tight_layout()

    return sae 
    
compare([K, tau], plot=True) # call the function with the initial values

### Optimizing Parameters

In [ ]:
from scipy.optimize import fmin #old minimizer
print("The original K = ", K, "Tau = ", tau)

p = fmin(compare, [K, tau]) # compare returns the SAE which fmin minimizes by changing K and tau
compare(p, plot=True)
print(p)

# scipy says that fmin is a legacy method and that new scripts should move to minimize instead, it works almost the same way
from scipy.optimize import minimize # new minimizer
# reset K and Tau
K = 0.6548 # °C/% U1
tau = 176.5 # sec
print("New Minimizer:")
p = minimize(compare, [K, tau])
print(p.x) # p.x is the array for the solutions
print(p)
compare(p.x, plot=True)

## Task 3: Derive Expressions for tau and K

The energy ballance is (Eq.1) 

$$
\begin{align}
C_p\frac{dT_1}{dt} & = U_a(T_{amb} - T_1) + \alpha P_1u_1 \\
\end{align}
$$

rearranged

$$
\begin{align}
\frac{dT_1}{dt} & = \frac{U_a}{C_p}(T_{amb} - T_1) + \frac{\alpha P_1}{C_p}u_1 \\
\end{align}
$$

then take 

$$\begin{align} x & = T_1 - T_{amb} \end{align}$$

to get $$\begin{align} \frac{dx}{dt} & = \frac{-U_a}{C_p}x + \frac{\alpha P_1}{C_p}u_1\end{align}$$  

the gain/time-constant form is $$\begin{align} \tau\frac{dx}{dt} = -x + K \bar{u} \end{align}$$


which rearranges to 
$$\begin{align} \frac{dx}{dt} = -\frac{1}{\tau} x + \frac{K}{\tau} \bar{u} \end{align}$$

so $$\begin{align} \tau & = \frac{C_p}{U_a} \end{align}$$

and $$\begin{align} K & = \frac{\alpha P_1}{C_p} \tau = \frac{\alpha P_1}{C_p} \frac{C_p}{U_a} = \frac{\alpha P_1}{U_a}\end{align}$$

plugging in for $K$ 

$$\begin{align} K & = \frac{\alpha P_1}{U_a} = 0.6759 \frac{degC}{\%} = \frac{0.16 \frac{mW}{\% unitsP_1}}{U_a} 200 units P_1 \end{align}
\begin{align} U_a = 47.34 \frac{mW}{degC}\end{align} $$

and plugging in for $\tau$ 

$$\begin{align} \tau & = \frac{C_p}{U_a} = 180.7 s = \frac{C_p}{47.34 \frac{mW}{degC}}\end{align}
\begin{align} C_p & = 8554\frac{mJ}{degC} = 8.55 \frac{J}{degC} \end{align}$$

# Task 4: Issues writting python code to automatically fit a model to the step test
edited student responses

**How can you measure the quality of fit?**

Quality of fit can be judged by standard absolute error and/or sum of square error. One interesting aspect is that, in the graphs above, the error is clearly dependent on the time of the experiment. This may indicate problems with the model, as the error should be random across the data set, not systematic. At the start of the experiment this seems attributable to some time delay as the heater conducts heat to the sensor. The model could be improved by accounting for the heat transfer between the heater and sensor.

**How should one handle bad measurements?**

In this case there is some noise in the measurments. Minimizing absolute error rather than the square error will keep large noisy measurements from being disproportionately amplified

**What parameters should be fit?**

Tau and K (or $C_p$ and $U_a$). The ambient temperature and starting temperature are known because the system should have been started at a steady state which was the ambient temperature. Alpha was given and $P_1$ and $u_1$ are set at the outset of the experiment.

**How can you estimate uncertainty in the estimates of $C_p$ and $U_a$?**

You can estimate the uncertainty in $C_p$ and $U_a$ through running several experiments and examining the range of parameter values.

**Is it possible to estimate $U_a$ from a steady state experiment?**

Yes - If d$T_1$/dt in Eq.1 above is set to 0 then $T_1$ is the steady state temperature and Alpha, $P_1$ and $U_1$ are all known constants so you can solve for $U_a$

**Is is possible to estimate $C_p$ from a steady state experiment?**

No - In a similar manner to above, if d$T_1$/dt in Eq.1 above is set to 0 then $C_p$ is a a constant multiplier of a 0 term that can't be found